# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [33]:
from azureml.core import Workspace, Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Dataset 

import joblib


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [34]:
label = 'vfc_level2_3_name'

In [35]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required

subscription_id = 'b968fb36-f06a-4c76-a15f-afab68ae7667'
resource_group = 'aml-quickstarts-201550'
workspace_name = 'quick-starts-ws-201550'

workspace = Workspace(subscription_id, resource_group, workspace_name)



In [36]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required.

validation_data = Dataset.get_by_name(workspace, name='test_project').to_pandas_dataframe()
train_data = Dataset.get_by_name(workspace, name='train_project').to_pandas_dataframe()


In [ ]:
subset_mvp = ['All-round view : View/restricted panoramic view', 'Android Auto : Malfunction', 
'Apps (A4A) : Not user-friendly', 'Comfort Access : Not user-friendly', 'Connected App : Malfunction', 
'Convertible top / hardtop, open / close : Malfunction', 'Cushioning / damping vehicle : Malfunction',
 'Display Key, use : Malfunction', 'Driver profile : Not user-friendly', 'Drivetrain : Customer request: function',
  'Driving experience, configuration : Not user-friendly', 'Electric Range : Range, not OK', 
  'Exterior lighting : Light intensity not OK', 'Headrest, adjust : Malfunction', 'High voltage battery : Malfunction', 
  'Hood : Damage', 'Instrument cluster : Not user-friendly', 'Lane departure warning : Not user-friendly', 
  'Manage / connect mobile device (WiFi) : Not user-friendly', 'Navigation : Not user-friendly', 
  'Operate gear selector switch : Not user-friendly', 'Panorama glass roof : Damage', 'Parking Assistant : Not user-friendly',
   'RSC tire : Not user-friendly', 'Seat belts : Not (sufficiently) comfortable', 'Seat belts : Not user-friendly',
    'Seating, ventilate / heat : Not user-friendly', 'Side window, open / close : Malfunction', 
    'Steering wheel : Noise', 'Steering wheel : Not user-friendly', 'Touch screen, operation : Dirty contamination', 
    'Vehicle, climatization : Condensation', 'Voice control / speech recognition : Malfunction', 
    'Voice control / speech recognition : Not user-friendly', 'Wheels / tires : Damage', 'good',
     'speech input / speech recognition : Not user-friendly']

In [ ]:
import pandas as pd
goodones = [x for x in pd.unique(train_data[label].dropna()) if ('like' in x.lower()) or ('appeals' in x.lower()) and x != 'Ergonomics do not appeal'] + [x for x in pd.unique(validation_data[label].dropna()) if ('like' in x.lower()) or ('appeals' in x.lower()) and x != 'Ergonomics do not appeal' ] 
validation_data = validation_data[validation_data[label].isin(subset_mvp)].set_index('Column1')
featurecols = validation_data.columns.tolist()
featurecols.remove(label)
X_val = validation_data[featurecols]
y_val = validation_data[label].apply(lambda x : 'good' if x in goodones else x)

train_data = train_data[validation_data[label].isin(subset_mvp)].set_index('Column1')
X_train = train_data[featurecols]
y_train = train_data[label].apply(lambda x : 'good' if x in goodones else x)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
automl_config = AutoMLConfig(
    experiment_timeout_hours=1.0,
    task= 'classification',
    primary_metric= 'average_precision_score_weighted',
    enable_early_stopping= True,
    featurization='off',
    X =X_train,
    y = y_train, 
    X_valid = X_val, 
    y_valid = y_val,
    enable_voting_ensemble=False,
    enable_stack_ensemble=False,
    allowed_models=['BernoulliNaiveBayes', 'KNN', 'RandomForest', 'SVM', 'XGBoostClassifier'],
    y_min= 100,
    y_max= 1200,)

In [ ]:
# Choose a name for your CPU cluster
cluster_name = "theresa"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=workspace, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(workspace, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)



In [ ]:
# TODO: Submit your experiment
# choose a name for experiment
experiment_name = 'capstone_automl'

experiment=Experiment(workspace, experiment_name)

run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, best_model = run.get_output()


In [ ]:
from azureml.core.model import Model


best_run = hyperdriverun.get_best_run_by_primary_metric()

model = best_run.register_model(model_name='automl_randomforest', 
                           model_path='outputs/model/automl_randomforest.pkl',
                           model_framework=Model.Framework.SCIKITLEARN,
                           model_framework_version='0.19.1')
model.download(target_dir='.', exist_ok=True, exists_ok=None)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
